In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn import (metrics, linear_model, preprocessing)
from sklearn.model_selection import train_test_split 
import xgboost as xgb 
from matplotlib import pyplot
# feature selection    make sure to update xgboost package
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, RepeatedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import uniform,randint
from xgboost import plot_importance

In [2]:
def read_data(path): 
    df = pd.read_csv(path, na_values='NA')
    # Printing the dataswet shape 
    print ("features:") 
    print (list(df.columns)) 
    print ("row and column number") 
    print (df.shape) 
    print ("data / feature types:") 
    print (df.dtypes) 
    df_num = df.select_dtypes(include='number')
    df_cat = df.select_dtypes(include='object')
    print ("missing values:") 
    print (df.isnull().sum()) 
    return [df, df_num, df_cat] 

In [3]:
path = '.\..\data\creditscore.csv'
dflist = read_data(path)

features:
['ID', 'overdue', 'utilization', 'age', 'num30_59dayspastdue', 'debtratio', 'monthlyincome', 'numlinecredit', 'num90dayslate', 'nummortgate', 'num60_89dayspastdue', 'numdependents']
row and column number
(150000, 12)
data / feature types:
ID                       int64
overdue                  int64
utilization            float64
age                      int64
num30_59dayspastdue      int64
debtratio              float64
monthlyincome          float64
numlinecredit            int64
num90dayslate            int64
nummortgate              int64
num60_89dayspastdue      int64
numdependents          float64
dtype: object
missing values:
ID                         0
overdue                    0
utilization                0
age                        0
num30_59dayspastdue        0
debtratio                  0
monthlyincome          29731
numlinecredit              0
num90dayslate              0
nummortgate                0
num60_89dayspastdue        0
numdependents           3924
d

In [5]:
scorecard = dflist[0]
predictor = list(scorecard.columns)


# even though xgboost does not need missimg treatment, we still do that
# since we need to analyze data patterns, such as correlation..
scorecard['monthlyincome_miss']=pd.isnull(scorecard['monthlyincome'])+0 # Si le añado un 0 significa que el True/False se convierte en su equivalente binario 1/0
scorecard['numdependents_miss']=pd.isnull(scorecard['numdependents'])+0
# fill missing with mean value
scorecard = scorecard.fillna(scorecard.mean())

In [6]:
varnamelist=list(scorecard.columns)
varnamelist.remove('ID')
varnamelist.remove('overdue')
Y = scorecard.overdue
Y.mean()
Y.value_counts()

corr=list()   # define an empty list
# loop for all independent variables
for vname in varnamelist: 
        X = scorecard[vname]              # loop all the variables in the data frame
        C = np.corrcoef(X, Y)             # calculate correlation matrix
        beta=np.round(C[1, 0],3)
        corr=corr+[beta]                # gather all correlation coefficients in a list


In [8]:

# change column names
corrdf = pd.DataFrame({'varname': varnamelist, 'correlation': corr})
corrdf['abscorr'] = np.abs(corrdf['correlation'])

# sort absolute value of correlation in descending order
corrdf.sort_values(['abscorr'], ascending=False, inplace=True)
seq = range(1,len(corrdf)+1)
corrdf['order']=seq  # add a sequential number column

corrdf

,varname,correlation,abscorr,order
2,num30_59dayspastdue,0.126,0.126,1
6,num90dayslate,0.117,0.117,2
1,age,-0.115,0.115,3
8,num60_89dayspastdue,0.102,0.102,4
9,numdependents,0.046,0.046,5
5,numlinecredit,-0.030,0.030,6
10,monthlyincome_miss,-0.021,0.021,7
4,monthlyincome,-0.018,0.018,8
11,numdependents_miss,-0.014,0.014,9
3,debtratio,-0.008,0.008,10


In [9]:
# get one hot features for num30_59dayspastdue, num90dayslate and num60_89dayspastdue 
scorecard.num90dayslate.value_counts()

names = ['num30_59dayspastdue', 'num90dayslate', 'num60_89dayspastdue']
for it in names:
    df_1 = pd.get_dummies(scorecard[it])
    colnames = list(df_1.columns)
    colnames_new = [it + "_" + str(colname) for colname in colnames]
    df_1.columns = colnames_new
    scorecard = pd.concat([scorecard, df_1], axis=1)

list(scorecard.columns)

predictor = list(scorecard.columns)

for s in ['overdue', 'ID']:
    predictor.remove(s)
    
print (predictor)

X = scorecard[predictor]
y = scorecard.overdue

['utilization', 'age', 'num30_59dayspastdue', 'debtratio', 'monthlyincome', 'numlinecredit', 'num90dayslate', 'nummortgate', 'num60_89dayspastdue', 'numdependents', 'monthlyincome_miss', 'numdependents_miss', 'num30_59dayspastdue_0', 'num30_59dayspastdue_1', 'num30_59dayspastdue_2', 'num30_59dayspastdue_3', 'num30_59dayspastdue_4', 'num30_59dayspastdue_5', 'num30_59dayspastdue_6', 'num30_59dayspastdue_7', 'num30_59dayspastdue_8', 'num30_59dayspastdue_9', 'num30_59dayspastdue_10', 'num30_59dayspastdue_11', 'num30_59dayspastdue_12', 'num30_59dayspastdue_13', 'num30_59dayspastdue_96', 'num30_59dayspastdue_98', 'num90dayslate_0', 'num90dayslate_1', 'num90dayslate_2', 'num90dayslate_3', 'num90dayslate_4', 'num90dayslate_5', 'num90dayslate_6', 'num90dayslate_7', 'num90dayslate_8', 'num90dayslate_9', 'num90dayslate_10', 'num90dayslate_11', 'num90dayslate_12', 'num90dayslate_13', 'num90dayslate_14', 'num90dayslate_15', 'num90dayslate_17', 'num90dayslate_96', 'num90dayslate_98', 'num60_89dayspa

In [13]:
import time
start_time = time.time()

# Cross-validation
NFOLDS = 5
kf = KFold(n_splits = NFOLDS, shuffle = True)

for tr_idx, val_idx in kf.split(X, y):

    clf = xgb.XGBClassifier(
    learning_rate =0.03,
    n_estimators = 1200,
    max_depth = 7,
    objective= 'binary:logistic',
    min_child_weight=5, 
    gamma=0.05, 
    subsample=0.8, 
    colsample_bytree=0.8,
    seed = 12,
    tree_method = 'hist'
  )
    
    X_tr, X_vl = X.iloc[tr_idx, :], X.iloc[val_idx, :]
    y_tr, y_vl = y.iloc[tr_idx], y.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    y_pred_train0 = clf.predict(X_vl)
    accuracy = accuracy_score(y_vl, y_pred_train0)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

    print("Process finished --- %s seconds ---" % (time.time() - start_time))

ROC AUC 0.8567698536061586
Accuracy: 93.64%
Process finished --- 11.461010456085205 seconds ---
ROC AUC 0.8615049600997617
Accuracy: 93.70%
Process finished --- 22.243997812271118 seconds ---
ROC AUC 0.8623139225706689
Accuracy: 93.46%
Process finished --- 33.155399322509766 seconds ---
ROC AUC 0.8573203914233312
Accuracy: 93.66%
Process finished --- 43.548136711120605 seconds ---
ROC AUC 0.8546381002091002
Accuracy: 93.64%
Process finished --- 53.57620286941528 seconds ---


In [14]:
import time
start_time = time.time()

# Cross-validation
NFOLDS = 5
kf = KFold(n_splits = NFOLDS, shuffle = True)

for tr_idx, val_idx in kf.split(X, y):

    clf = xgb.XGBClassifier(
    learning_rate =0.03,
    n_estimators = 1200,
    max_depth = 7,
    objective= 'binary:logistic',
    min_child_weight=5, 
    gamma=0.05, 
    subsample=0.8, 
    colsample_bytree=0.8,
    seed = 12,
    tree_method = 'gpu_hist'
  )
    
    X_tr, X_vl = X.iloc[tr_idx, :], X.iloc[val_idx, :]
    y_tr, y_vl = y.iloc[tr_idx], y.iloc[val_idx]
    clf.fit(X_tr, y_tr)
    y_pred_train = clf.predict_proba(X_vl)[:,1]
    
    print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
    
    y_pred_train0 = clf.predict(X_vl)
    accuracy = accuracy_score(y_vl, y_pred_train0)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Process finished --- %s seconds ---" % (time.time() - start_time))

ROC AUC 0.857389216215282
Accuracy: 93.62%
Process finished --- 8.203361511230469 seconds ---
ROC AUC 0.8614314700959277
Accuracy: 93.60%
Process finished --- 16.21073889732361 seconds ---
ROC AUC 0.8558097468570025
Accuracy: 93.44%
Process finished --- 24.11885714530945 seconds ---
ROC AUC 0.8608536165882742
Accuracy: 93.66%
Process finished --- 32.02174210548401 seconds ---
ROC AUC 0.8591300215754338
Accuracy: 93.74%
Process finished --- 40.13109993934631 seconds ---


#### Using native booster

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split( 
        X, y, test_size = 0.15, random_state = 122) 


params = {'learning_rate' : 0.03,
    'max_depth' : 7,
    'objective' : 'binary:logistic',
    'min_child_weight' : 5, 
    'gamma' : 0.05, 
    'subsample': 0.8, 
    'colsample_bytree': 0.8,
    'seed' : 12,
    'eval_metric' : 'auc',
    'tree_method' : 'gpu_hist',
    'sampling_method': 'gradient_based'
}


dtrain = xgb.DMatrix(data=X_train.values,
                    feature_names=X_train.columns,
                    label=y_train.values)

dvalid = xgb.DMatrix(data=X_valid.values,
                    feature_names=X_valid.columns,
                    label=y_valid.values)

mod = xgb.train(params=params,
                dtrain=dtrain,
                num_boost_round = 1200, # Número máximo de iteraciones que el algoritmo va a correr
                early_stopping_rounds=50, # Número de rondas hacia el futuro de la que ve hasta el momento para determinar si parar el entrenamiento o no
                evals=[(dvalid,'valid'), (dtrain,'train')],
                verbose_eval=20)

[0]	valid-auc:0.84696	train-auc:0.84722
[20]	valid-auc:0.86153	train-auc:0.86502
[40]	valid-auc:0.86270	train-auc:0.86724
[60]	valid-auc:0.86315	train-auc:0.86902
[80]	valid-auc:0.86426	train-auc:0.87128
[100]	valid-auc:0.86529	train-auc:0.87374
[120]	valid-auc:0.86574	train-auc:0.87639
[140]	valid-auc:0.86625	train-auc:0.87892
[160]	valid-auc:0.86651	train-auc:0.88116
[180]	valid-auc:0.86656	train-auc:0.88342
[200]	valid-auc:0.86660	train-auc:0.88543
[220]	valid-auc:0.86692	train-auc:0.88736
[240]	valid-auc:0.86680	train-auc:0.88893
[260]	valid-auc:0.86671	train-auc:0.89019
[280]	valid-auc:0.86654	train-auc:0.89136
[300]	valid-auc:0.86642	train-auc:0.89227
[320]	valid-auc:0.86631	train-auc:0.89349
[340]	valid-auc:0.86623	train-auc:0.89424
[360]	valid-auc:0.86617	train-auc:0.89512
[380]	valid-auc:0.86601	train-auc:0.89612
[400]	valid-auc:0.86599	train-auc:0.89710
[420]	valid-auc:0.86594	train-auc:0.89791
[440]	valid-auc:0.86586	train-auc:0.89869
[460]	valid-auc:0.86579	train-auc:0.8995

#### Usando 'cv' para cross-validation

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split( 
        X, y, test_size = 0.27, random_state = 122) 


dtrain = xgb.DMatrix(data=X_train.values,
                    feature_names=X_train.columns,
                    label=y_train.values)

dvalid = xgb.DMatrix(data=X_valid.values,
                    feature_names=X_valid.columns,
                    label=y_valid.values)

xgb_params = {'learning_rate' : 0.03,
    'max_depth' : 7,
    'objective' : 'binary:logistic',
    'min_child_weight' : 5, 
    'gamma' : 0.05, 
    'subsample': 0.8, 
    'colsample_bytree': 0.8,
    'seed' : 12
}

eval_cv = xgb.cv(xgb_params, dtrain, num_boost_round=1200, nfold = 5,
            metrics='auc', early_stopping_rounds=80)
        
clf = xgb.XGBClassifier(
    learning_rate =0.03,
    n_estimators = 1200,
    max_depth = 7,
    objective= 'binary:logistic',
    min_child_weight=5, 
    gamma=0.05, 
    subsample=0.8, 
    colsample_bytree=0.8,
    seed = 12,
    tree_method='gpu_hist',
    sampling_method='gradient_based')
    
    
clf.set_params(n_estimators = eval_cv.shape[0])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.05, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=216, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [22]:
clf.set_params(n_estimators = eval_cv.shape[0])
        
clf.fit(X_train, y_train, eval_metric='auc', verbose=True)
        
predictions_prob = clf.predict_proba(X_valid)[:,1]      
    
print('ROC AUC {}'.format(roc_auc_score(y_valid, predictions_prob)))


#  we use the following ways
imp_gain = clf.get_booster().get_score(importance_type="gain")
imp_weight = clf.get_booster().get_score(importance_type="weight")

e:\udemy\xgboost\xgboost_env\Lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


ROC AUC 0.8654487321857303


In [23]:
print(imp_gain)
print(imp_weight)

{'utilization': 26.86141586303711, 'age': 5.045316696166992, 'num30_59dayspastdue': 34.05958938598633, 'debtratio': 4.193957805633545, 'monthlyincome': 3.8325703144073486, 'numlinecredit': 4.916317939758301, 'num90dayslate': 115.24346160888672, 'nummortgate': 6.071907997131348, 'num60_89dayspastdue': 19.200992584228516, 'numdependents': 3.367438316345215, 'monthlyincome_miss': 4.214170932769775, 'numdependents_miss': 2.8333280086517334, 'num30_59dayspastdue_0': 40.428749084472656, 'num30_59dayspastdue_1': 5.541392803192139, 'num30_59dayspastdue_2': 4.4015116691589355, 'num30_59dayspastdue_3': 3.854369878768921, 'num30_59dayspastdue_4': 3.5719306468963623, 'num30_59dayspastdue_5': 4.550775527954102, 'num30_59dayspastdue_6': 3.0224950313568115, 'num30_59dayspastdue_98': 0.5659983158111572, 'num90dayslate_0': 131.66262817382812, 'num90dayslate_1': 12.234856605529785, 'num90dayslate_2': 12.029308319091797, 'num90dayslate_3': 12.055765151977539, 'num90dayslate_4': 5.5554070472717285, 'num90